In [9]:
import pickle

def loadDataPickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [10]:
movies = loadDataPickle('MovieDatasetCleaned.pickle')

In [11]:
import pandas as pd

df = pd.DataFrame(movies)

df.head()

,Title,Produced by,Cinematography,Release dates,Running time,Country,Languages,Running Time (int),Budget (float),Box Office (float),...,Music by,Production companies,Countries,Budget,Box office,Based on,Story by,Narrated by,Layouts by,Animation by
0,Garbage Dreams,Mai Iskander,Mai Iskander,"[April 3, 2009 ( Phoenix ), July 31, 2009 (Uni...",79 minutes,United States,Egyptian Arabic,79.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sweetgrass,Ilisa Barbash,Lucien Castaing- Taylor,NaN,101 minutes,United States,NaN,101.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Daybreakers,"[Chris Brown, Sean Furst, Bryan Furst]",Ben Nott,"[September 11, 2009 ( TIFF ), January 8, 2010 ...",98 minutes,NaN,NaN,98.0,20000000.0,51400000.0,...,Christopher Gordon,"[Lionsgate, Screen Australia, Pictures in Para...",Australia United States,$20 million,$51.4 million,NaN,NaN,NaN,NaN,NaN
3,Leap Year,"[Gary Barber, Chris Bender, Roger Birnbaum, Jo...",Newton Thomas Sigel,"[January 6, 2010 ( New York City ), January 8,...",100 minutes,NaN,NaN,100.0,19000000.0,32600000.0,...,Randy Edelman,Spyglass Entertainment,United States Ireland,$19 million,$32.6 million,NaN,NaN,NaN,NaN,NaN
4,Youth in Revolt,David Permut,Chuy Chávez,"[October 19, 2009 ( DAFF ), January 8, 2010 (U...",89 minutes,United States,NaN,89.0,18000000.0,19700000.0,...,John Swihart,"[Permut, Shangri-La Entertainment]",NaN,$18 million,$19.7 million,"[Youth in Revolt, C.D. Payne]",NaN,NaN,NaN,NaN


In [12]:
# See All Columns in The Dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3254 entries, 0 to 3253
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Title                    3254 non-null   object        
 1   Produced by              3224 non-null   object        
 2   Cinematography           3092 non-null   object        
 3   Release dates            2089 non-null   object        
 4   Running time             3207 non-null   object        
 5   Country                  2574 non-null   object        
 6   Languages                205 non-null    object        
 7   Running Time (int)       3207 non-null   float64       
 8   Budget (float)           2131 non-null   float64       
 9   Box Office (float)       2551 non-null   float64       
 10  Release Date (datetime)  3222 non-null   datetime64[ns]
 11  Directed by              3226 non-null   object        
 12  Distributed by           3204 non-

In [13]:
# Drop Unnecessary Columns
df.drop(['Release dates', 'Release date', 'Countries', 'Running time', 'Languages', 'Budget', 'Box office', 'Narrated by', 'Layouts by', 'Animation by'], axis=1, inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3254 entries, 0 to 3253
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Title                    3254 non-null   object        
 1   Produced by              3224 non-null   object        
 2   Cinematography           3092 non-null   object        
 3   Country                  2574 non-null   object        
 4   Running Time (int)       3207 non-null   float64       
 5   Budget (float)           2131 non-null   float64       
 6   Box Office (float)       2551 non-null   float64       
 7   Release Date (datetime)  3222 non-null   datetime64[ns]
 8   Directed by              3226 non-null   object        
 9   Distributed by           3204 non-null   object        
 10  Language                 3024 non-null   object        
 11  Written by               3175 non-null   object        
 12  Starring                 3185 non-

In [15]:
df.head()

,Title,Produced by,Cinematography,Country,Running Time (int),Budget (float),Box Office (float),Release Date (datetime),Directed by,Distributed by,Language,Written by,Starring,Edited by,Music by,Production companies,Based on,Story by
0,Garbage Dreams,Mai Iskander,Mai Iskander,United States,79.0,NaN,NaN,2009-04-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sweetgrass,Ilisa Barbash,Lucien Castaing- Taylor,United States,101.0,NaN,NaN,2009-02-04,Lucien Castaing-Taylor,Cinema Guild,English,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Daybreakers,"[Chris Brown, Sean Furst, Bryan Furst]",Ben Nott,NaN,98.0,20000000.0,51400000.0,2009-09-11,The Spierig Brothers,Hoyts Distribution,English,The Spierig Brothers,"[Ethan Hawke, Willem Dafoe, Claudia Karvan, Mi...",Matt Villa,Christopher Gordon,"[Lionsgate, Screen Australia, Pictures in Para...",NaN,NaN
3,Leap Year,"[Gary Barber, Chris Bender, Roger Birnbaum, Jo...",Newton Thomas Sigel,NaN,100.0,19000000.0,32600000.0,2010-01-06,Anand Tucker,"[Universal Pictures (United States), Optimum R...",English,"[Harry Elfont, Deborah Kaplan]","[Amy Adams, Matthew Goode, Adam Scott, John Li...",Nick Moore,Randy Edelman,Spyglass Entertainment,NaN,NaN
4,Youth in Revolt,David Permut,Chuy Chávez,United States,89.0,18000000.0,19700000.0,2009-10-19,Miguel Arteta,Dimension Films,English,Gustin Nash,"[Michael Cera, Portia Doubleday, Jean Smart, M...",Pamela Martin,John Swihart,"[Permut, Shangri-La Entertainment]","[Youth in Revolt, C.D. Payne]",NaN


In [16]:
endColumn = ['Budget (float)', 'Box Office (float)']
df = df[[column for column in df if column not in endColumn] + [column for column in endColumn if column in df]]
df.head()

,Title,Produced by,Cinematography,Country,Running Time (int),Release Date (datetime),Directed by,Distributed by,Language,Written by,Starring,Edited by,Music by,Production companies,Based on,Story by,Budget (float),Box Office (float)
0,Garbage Dreams,Mai Iskander,Mai Iskander,United States,79.0,2009-04-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sweetgrass,Ilisa Barbash,Lucien Castaing- Taylor,United States,101.0,2009-02-04,Lucien Castaing-Taylor,Cinema Guild,English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Daybreakers,"[Chris Brown, Sean Furst, Bryan Furst]",Ben Nott,NaN,98.0,2009-09-11,The Spierig Brothers,Hoyts Distribution,English,The Spierig Brothers,"[Ethan Hawke, Willem Dafoe, Claudia Karvan, Mi...",Matt Villa,Christopher Gordon,"[Lionsgate, Screen Australia, Pictures in Para...",NaN,NaN,20000000.0,51400000.0
3,Leap Year,"[Gary Barber, Chris Bender, Roger Birnbaum, Jo...",Newton Thomas Sigel,NaN,100.0,2010-01-06,Anand Tucker,"[Universal Pictures (United States), Optimum R...",English,"[Harry Elfont, Deborah Kaplan]","[Amy Adams, Matthew Goode, Adam Scott, John Li...",Nick Moore,Randy Edelman,Spyglass Entertainment,NaN,NaN,19000000.0,32600000.0
4,Youth in Revolt,David Permut,Chuy Chávez,United States,89.0,2009-10-19,Miguel Arteta,Dimension Films,English,Gustin Nash,"[Michael Cera, Portia Doubleday, Jean Smart, M...",Pamela Martin,John Swihart,"[Permut, Shangri-La Entertainment]","[Youth in Revolt, C.D. Payne]",NaN,18000000.0,19700000.0


In [20]:
# Convert Running Time to Integer And Skip Nan Values
df['Running Time (int)'] = df['Running Time (int)'].astype('Int64')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3254 entries, 0 to 3253
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Title                    3254 non-null   object        
 1   Produced by              3224 non-null   object        
 2   Cinematography           3092 non-null   object        
 3   Country                  2574 non-null   object        
 4   Running Time (int)       3207 non-null   Int64         
 5   Release Date (datetime)  3222 non-null   datetime64[ns]
 6   Directed by              3226 non-null   object        
 7   Distributed by           3204 non-null   object        
 8   Language                 3024 non-null   object        
 9   Written by               3175 non-null   object        
 10  Starring                 3185 non-null   object        
 11  Edited by                3194 non-null   object        
 12  Music by                 3067 non-

In [22]:
df.head()

,Title,Produced by,Cinematography,Country,Running Time (int),Release Date (datetime),Directed by,Distributed by,Language,Written by,Starring,Edited by,Music by,Production companies,Based on,Story by,Budget (float),Box Office (float)
0,Garbage Dreams,Mai Iskander,Mai Iskander,United States,79,2009-04-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sweetgrass,Ilisa Barbash,Lucien Castaing- Taylor,United States,101,2009-02-04,Lucien Castaing-Taylor,Cinema Guild,English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Daybreakers,"[Chris Brown, Sean Furst, Bryan Furst]",Ben Nott,NaN,98,2009-09-11,The Spierig Brothers,Hoyts Distribution,English,The Spierig Brothers,"[Ethan Hawke, Willem Dafoe, Claudia Karvan, Mi...",Matt Villa,Christopher Gordon,"[Lionsgate, Screen Australia, Pictures in Para...",NaN,NaN,20000000.0,51400000.0
3,Leap Year,"[Gary Barber, Chris Bender, Roger Birnbaum, Jo...",Newton Thomas Sigel,NaN,100,2010-01-06,Anand Tucker,"[Universal Pictures (United States), Optimum R...",English,"[Harry Elfont, Deborah Kaplan]","[Amy Adams, Matthew Goode, Adam Scott, John Li...",Nick Moore,Randy Edelman,Spyglass Entertainment,NaN,NaN,19000000.0,32600000.0
4,Youth in Revolt,David Permut,Chuy Chávez,United States,89,2009-10-19,Miguel Arteta,Dimension Films,English,Gustin Nash,"[Michael Cera, Portia Doubleday, Jean Smart, M...",Pamela Martin,John Swihart,"[Permut, Shangri-La Entertainment]","[Youth in Revolt, C.D. Payne]",NaN,18000000.0,19700000.0


In [ ]:
df.to_csv('WikipediaMovieDataset.csv', index=False)